#### Simple Gen AI APP Using Langchain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [2]:
## Data Ingestion--From the website we need to scrape the data
#USER_AGENT environment variable not set, consider setting it to identify your requests. we get this error sometimes
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
loader

In [10]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content='\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Cloud)AdministrationSelf-hostingReferenceQuick StartOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain\'s open source frameworks is not necessary - LangSmith works on it

In [11]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [12]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content="Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick 

In [17]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()
#converted text into embeddings

In [18]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [19]:
vectorstoredb

In [ ]:
## Query From a vector db
query="LangSmith has two usage limits: total traces and extended"
#taken this query from website
result=vectorstoredb.similarity_search(query)
result[0].page_content

"4. Log your first trace\u200b\nLangSmith + LangChain OSSYou don't need to use the LangSmith SDK directly if your application is built on LangChain/LangGraph (either Python and JS).See the how-to guide for tracing with LangChain here.\nWe provide multiple ways to log traces to LangSmith. Below, we'll highlight\nhow to use traceable(). See more on the Annotate code for tracing page."

In [21]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
## Retrieval Chain, Document chain
#read more about this retrieval chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), config={'run_name': 'format_inputs'})
| ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F6FF6107C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F6FF6138B0>, root_client=<openai.OpenAI object at 0x000001F6FF62FEB0>, root_async_client=<openai.AsyncOpenAI object at 0x000001F6FF610070>, model_name='gpt-4o', openai_api_key=SecretStr('**********'), openai_proxy='')
| StrOutputParser(), config={'run_name': 'stuff_documents_chain'})

In [23]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. ")]
})

'LangSmith has two usage limits: one for total traces and another for extended traces.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [24]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [ ]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
# vwctordb has all vectors. REtriveer can be considered as interface, its responsibility is, if any one gives input, this retriever, is a way of getting the 
#data from vectordb
#document_chain abve will help give the context


In [26]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001F6FF662AD0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001F6FF6107C0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001F

In [27]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'To log your first trace with LangSmith, follow these steps:\n\n1. **Install LangSmith**: Use the following commands for installation:\n   - For Python: `pip install -U langsmith openai`\n   - For TypeScript: `yarn add langsmith openai`\n\n2. **Create an API key**: Go to the Settings page on the LangSmith platform and click "Create API Key" to generate your API key.\n\n3. **Set up your environment**: Configure your environment by exporting the necessary environment variables. Use the following commands in your shell:\n   ```bash\n   export LANGCHAIN_TRACING_V2=true\n   export LANGCHAIN_API_KEY=<your-api-key>\n   export OPENAI_API_KEY=<your-openai-api-key>  # This is optional if you\'re using OpenAI API\n   ```\n\n4. **Log your first trace**: After running your evaluation, click the link printed out by your evaluation run to access the LangSmith experiments UI. There you can explore the results of your evaluation.\n\nThis process allows you to closely monitor and evaluate your LLM appli

In [29]:

response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content="4. Log your first trace\u200b\nLangSmith + LangChain OSSYou don't need to use the LangSmith SDK directly if your application is built on LangChain/LangGraph (either Python and JS).See the how-to guide for tracing with LangChain here.\nWe provide multiple ways to log traces to LangSmith. Below, we'll highlight\nhow to use traceable(). See more on the Annotate code for tracing page."),
  Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get star

In [30]:
response['context']

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain's open source frameworks is not necessary - LangSmith works on its own!", 'language': 'en'}, page_content="4. Log your first trace\u200b\nLangSmith + LangChain OSSYou don't need to use the LangSmith SDK directly if your application is built on LangChain/LangGraph (either Python and JS).See the how-to guide for tracing with LangChain here.\nWe provide multiple ways to log traces to LangSmith. Below, we'll highlight\nhow to use traceable(). See more on the Annotate code for tracing page."),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': "LangSmith is a platform for bu